<h1><center>TP1 - Ex.2</center></h1>
<p><center>Fevereiro 20, 2024</center></p>


### Estruturas Criptográficas

PG53886, Ivo Miguel Alves Ribeiro

A95323, Henrique Ribeiro Fernandes

#### Trabalho Pratico 1 Exercicio 2

2. Use o “package” Cryptography para

    a. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha2

**Cifragem/Decifragem da Mensagem com “Tweakable Block Ciphers” e AEAD:**
   - Para cifrar uma mensagem, inicialmente realizamos a autenticação da mesma juntamente com os metadados. 
   - Em seguida, aplicamos o padding para garantir que todos os blocos tenham o mesmo tamanho e evitem blocos incompletos. 
   - Posteriormente, dividimos a mensagem em blocos de tamanho fixo e ciframos cada bloco individualmente. Neste processo, utilizamos o AES-256 como um "tweakable block cipher". Isso significa que o algoritmo AES-256 é utilizado com um tweak (ou ajuste) específico, que é um valor único para cada bloco, garantindo a segurança adicional.
   - Por fim, os blocos cifrados são concatenados para formar a mensagem cifrada final.
   - Enquanto que para a decifragem foi o percurso inverso








In [1]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

def hash_func(lenght,message):
    hkdf = HKDF(
            algorithm=hashes.SHA256(),
            length=lenght,
            salt=None,
            info=b'tweak',
            backend=default_backend()
    )

    new_hash = hkdf.derive(message)
    
    return new_hash

def aes_encrypt_block(block,key,tweak):
        aes_cipher = Cipher(algorithms.AES256(key),mode=modes.CTR(tweak),backend=default_backend())
        encryptor_aes = aes_cipher.encryptor()
        ciphertext_aes = encryptor_aes.update(block) + encryptor_aes.finalize()
        return ciphertext_aes


def encrypt(plaintext,metadados,key):
    padder = padding.PKCS7(256).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = b''

    tweak = os.urandom(32)

    for i in range(0, len(padded_plaintext), 32):
        block = padded_plaintext[i:i+32]

        new_tweak = hash_func(16,key + tweak*i)
        ciphertext_block = aes_encrypt_block(block,key,new_tweak)

        ciphertext += ciphertext_block

    new_tweak = hash_func(16,key + tweak*i)

    autenticaçao = hmac.HMAC(new_tweak, hashes.SHA256())
    autenticaçao.update(metadados+plaintext)
    tag = autenticaçao.finalize()

    t_meta = len(metadados)
    return t_meta.to_bytes(2, 'big') + metadados+ tweak + tag + ciphertext


def aes_decrypt_block(block,key,tweak):

    aes_cipher = Cipher(algorithms.AES256(key),mode=modes.CTR(tweak),backend=default_backend())
    decryptor_aes = aes_cipher.decryptor()
    plaintext_aes = decryptor_aes.update(block) + decryptor_aes.finalize()
    return plaintext_aes

def decrypt(ciphertext,key):
    plaintext = b''
    
    metadados, tweak, tag, ciphertext = serialize_2(ciphertext)

    # Dividindo o texto cifrado em blocos de 16 bytes (128 bits)
    for i in range(0, len(ciphertext), 32):
        block = ciphertext[i:i+32]

        new_tweak = hash_func(16,key + tweak*i)
        # Decifrar o bloco usando AES
        plaintext_block = aes_decrypt_block(block,key,new_tweak)
        
        # Adicionar o bloco decifrado ao texto plano final
        plaintext += plaintext_block
    
    new_tweak = hash_func(16,key + tweak*i)
    
    unpadder = padding.PKCS7(256).unpadder()
    unpadded_plaintext = unpadder.update(plaintext) + unpadder.finalize()

    autenticaçao = hmac.HMAC(new_tweak, hashes.SHA256())
    autenticaçao.update(metadados+unpadded_plaintext)
    new_tag = autenticaçao.finalize()

    if tag == new_tag:
        print("(Receiber) Autenticaçao feita")
        return metadados ,unpadded_plaintext
    else : 
        print("(Receiber) Erro na autenticaçao")
        return None ,None


def serialize_2(data):
    offset0 = 0 
    offset1 = 2
    t_meta = int.from_bytes(data[offset0:offset1], "big")
    offset0 = offset1
    offset1 = offset1 + t_meta
    metadados = data[offset0:offset1]
    offset0 = offset1
    offset1 = offset1 + 32
    tweak = data[offset0:offset1]
    offset0 = offset1
    offset1 = offset1 + 32
    tag = data[offset0:offset1]
    offset0 = offset1
    data = data[offset0:]
    return (metadados, tweak, tag, data)

    
2. Use o “package” Cryptography para

    b . Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

### Generate keys
Para o receiver e para o emitter, criamos duas chaves privadas e duas públicas, um par do tipo x448 e outro par do tipo ed448. Em seguida, autenticamos a chave pública do x448 através das chaves ed448, resultando em uma assignature.

### Troca de chaves
Para a troca de chaves entre o receiver e o emitter, enviamos uma mensagem contendo as chaves públicas e a assignature. Ao receber a mensagem, autenticamos a chave pública do x448 através da chave pública do ed448 e da assignature. Com a chave pública x448 autenticada, realizamos uma troca e criamos uma shared_key. Posteriormente, realizamos um hash na chave para garantir que tenha os bytes desejados, neste caso, 32 bytes.

In [2]:
from cryptography.hazmat.primitives.asymmetric import x448
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey,Ed448PublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

def generate_key_2():
    #chaves x448
    x448_private_key = x448.X448PrivateKey.generate()
    x448_public_key = x448_private_key.public_key().public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    
    #chaves ed448
    ed448_private_key = Ed448PrivateKey.generate()
    signature = ed448_private_key.sign(x448_public_key)
    ed448_public_key = ed448_private_key.public_key()

    ed448_public_key.verify(signature, x448_public_key)
    ed448_public_key =ed448_public_key.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )

    return x448_private_key,x448_public_key,ed448_public_key,signature

def send_handshake(x448_public_key,ed448_public_key,signature,writter):
    writter.write(x448_public_key + ed448_public_key +signature)

def receive_handshake(x448_private_key,data):
    peer_x448_public_key = data[:56]
    peer_ed448_public_key= data[56:113]
    peer_ed448_public_key = Ed448PublicKey.from_public_bytes(peer_ed448_public_key)
    signature = data[113:]
    peer_ed448_public_key.verify(signature, peer_x448_public_key)

    peer_x448_public_key = x448.X448PublicKey.from_public_bytes(peer_x448_public_key)
       
    shared_key = x448_private_key.exchange(peer_x448_public_key)
    
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'key derivation',
        backend=default_backend()
    )

    print("Um Handchake concluido")
    aes_key = hkdf.derive(shared_key)
    
    return aes_key

### Emitter
Este emitter tem a função de cifrar uma mensagem e enviá-la usando as funções criadas anteriormente.

In [3]:
import asyncio

async def emmiter_2(metadados,message):
    x448_private_key , x448_public_key,ed448_public_key ,signature = generate_key_2()

    reader, writer = await asyncio.open_connection('127.0.0.1', 12368)

    send_handshake(x448_public_key,ed448_public_key,signature,writer)
    data = await reader.read(400)
    shared_key = receive_handshake(x448_private_key,data)
  
    data = encrypt(message,metadados,shared_key)

    print(f"(Emitter) Mensagem enviada: {message}\n(Emitter) Metadados enviados: {metadados}")
    print("(Emitter) data enviada:", data.hex())
    writer.write(data)
    await writer.drain()

    writer.close()
    await writer.wait_closed()

### Receiver
Este receiber tem a função de receber uma mensagem e decifrá-la usando as funções criadas anteriormente.

In [4]:
import asyncio

async def recive_msg_2(reader, writer, x448_private_key , x448_public_key,ed448_public_key ,signature):

    addr = writer.get_extra_info('peername')

    send_handshake(x448_public_key,ed448_public_key,signature,writer)
    data = await reader.read(400)
    shared_key = receive_handshake(x448_private_key,data)

    metadados = b"criado para fins de teste"
    
    data = await reader.read(400)
    if data:
        print("(Receiver) Pacote recebido: ", data.hex() , " from ", addr)

        metadados,message = decrypt(data,shared_key)

        print(f"(Receiver) Mensagem decifrada: {message}\n(Receiver) Metadados recebidos: {metadados}")
    writer.close()


async def receiver_2():
    x448_private_key , x448_public_key,ed448_public_key ,signature = generate_key_2()

    server = await asyncio.start_server(
        lambda r, w: recive_msg_2(r, w,x448_private_key , x448_public_key,ed448_public_key ,signature), '127.0.0.1', 12368)
    addr = server.sockets[0].getsockname()
    print(f"(Receiver) Serving on {addr}")
    await asyncio.sleep(3)
    server.close()
    await server.wait_closed()

### Main
A nossa main inicia o receiver e o emitter dando somente ao emitter uma mensagem e os seus metadados

In [5]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

metadados = b"criado para fins de teste"
message = b"Hello, server!"

async def main():
    task1 = asyncio.create_task(receiver_2())
    task2 = asyncio.create_task(emmiter_2(metadados,message))
    await asyncio.gather(task1, task2)

await main()

(Receiver) Serving on ('127.0.0.1', 12368)


Um Handchake concluido
Um Handchake concluido
(Emitter) Mensagem enviada: b'Hello, server!'
(Emitter) Metadados enviados: b'criado para fins de teste'
(Emitter) data enviada: 001963726961646f20706172612066696e732064652074657374651088d80e1dc665efae0e96b7a2c51dc08b38c087f04bb0f5b7f3fc6f395c65914b69d8abc3644cbee784ed7e89d2ddf35cd8ff9928f5c97d901c955c5c400864f72894acf934e1a6d798023f327c953a7d735463f8aa9b01fac9bb9c3c7f8d1c
(Receiver) Pacote recebido:  001963726961646f20706172612066696e732064652074657374651088d80e1dc665efae0e96b7a2c51dc08b38c087f04bb0f5b7f3fc6f395c65914b69d8abc3644cbee784ed7e89d2ddf35cd8ff9928f5c97d901c955c5c400864f72894acf934e1a6d798023f327c953a7d735463f8aa9b01fac9bb9c3c7f8d1c  from  ('127.0.0.1', 56184)
(Receiber) Autenticaçao feita
(Receiver) Mensagem decifrada: b'Hello, server!'
(Receiver) Metadados recebidos: b'criado para fins de teste'
